In [1]:
# section 1 load all the necessary modules and packages
import glob
import time
import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


# breaking up the global file into the regions

In [2]:
## open and save the basin shapefile into elements:
path = '/Volumes/F:/hydrography/HDMA/processed/'

shp = gpd.read_file(path+'hdma_global_catch_v2.gpkg')


warnings.simplefilter('ignore') # silent the warning
for i in np.array([1000000,2000000,3000000,4000000,5000000,6000000]):
    
    shp_sub = shp [shp['hruid']>=i]
    shp_sub = shp_sub [shp_sub['hruid']<(i+1000000)]
    shp_sub_temp = shp_sub.to_crs ("EPSG:6933")
    shp_sub ['area_org'] = shp_sub_temp.area
    shp_sub.to_file(path+'hdma_global_catch_v2_'+str(i)+'.gpkg')
    
warnings.simplefilter('default') # back to default




In [3]:
path = '/Volumes/F:/hydrography/HDMA/processed/'
folder_to_save = './shp_output/'
extension = ''

lakes = gpd.read_file(folder_to_save+'HydroLAKES_polys_v10_'+extension+'_final.shp')

for i in np.array([1000000,2000000,3000000,4000000,5000000,6000000]):
    
    # 
    basin = gpd.read_file(path+'hdma_global_catch_v2_'+str(i)+'.gpkg')
    
    #
    basin_corrected = gpd.overlay(basin,lakes, how='difference')
    basin_corrected = basin_corrected.dissolve(by=['hruid'])
    basin_corrected['hruid'] = basin_corrected.index
    basin_corrected = basin_corrected.reset_index(drop=True)
    basin_corrected_temp = basin_corrected.to_crs ("EPSG:6933")
    basin_corrected['area_cor'] = basin_corrected_temp.area
    basin_corrected['ID'] = basin_corrected['hruid']
    basin_corrected['lake'] = 0
    basin_corrected = basin_corrected.filter(['ID', 'area_cor','area_org', 'geometry', 'lake'])
    print(basin_corrected)
    print(len(basin_corrected))
    
    #
    lake_correction = gpd.overlay(lakes,basin, how='intersection')
    lake_ID_in = np.unique(lake_correction['Hylak_id'])
    lake_slice = lakes[lakes['Hylak_id'].isin(lake_ID_in)]
    lake_slice ['ID'] = lake_slice ['Hylak_id']
    lake_slice ['area_org'] = lake_slice ['Lake_area'] *1000000
    lake_slice ['area_cor'] = lake_slice ['area_org']
    lake_slice ['lake'] = 1
    lake_slice = lake_slice.filter(['ID', 'area_cor','area_org', 'geometry', 'lake'])
    print(lake_slice)
    print(len(lake_slice))
    
    # compile by the model simulation
    basin_corrected = basin_corrected.append(lake_slice)
    print(basin_corrected)
    print(len(basin_corrected))
    
    # save lake+corrected
    basin_corrected.to_file(folder_to_save+'HDMA_corrected_basin_HydroLAKES_polys_v10_'+\
                            extension+'_final_'+str(i)+'.shp')
    
    

            ID      area_cor      area_org  \
0      1000000  2.482575e+08  2.482575e+08   
1      1000001  2.492364e+08  2.492364e+08   
2      1000002  2.492635e+08  2.492635e+08   
3      1000003  2.478547e+08  2.478547e+08   
4      1000004  2.466977e+08  2.466977e+08   
...        ...           ...           ...   
76115  1076373  6.768024e+07  6.768024e+07   
76116  1076374  9.309675e+07  9.309675e+07   
76117  1076375  3.463953e+08  3.463953e+08   
76118  1076376  7.235799e+08  7.235799e+08   
76119  1076377  4.736172e+08  4.736172e+08   

                                                geometry  lake  
0      POLYGON ((145.55819 60.31653, 145.56236 60.316...     0  
1      POLYGON ((75.79417 15.78083, 75.79417 15.77833...     0  
2      POLYGON ((120.84083 29.39333, 120.84333 29.393...     0  
3      MULTIPOLYGON (((118.91583 32.75917, 118.91500 ...     0  
4      POLYGON ((135.25917 45.84083, 135.25917 45.839...     0  
...                                                  ... 

/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


           ID      area_cor      area_org  \
8     7000011  3.196785e+10  3.196785e+10   
9     7000012  1.671789e+10  1.671789e+10   
10    7000013  2.386591e+10  2.386591e+10   
82    7000091  4.908660e+09  4.908660e+09   
84    7000093  2.188580e+09  2.188580e+09   
...       ...           ...           ...   
4176  8394227  4.800000e+05  4.800000e+05   
4177  8396738  2.000000e+05  2.000000e+05   
4178  8396887  4.100000e+05  4.100000e+05   
4181  8398357  1.300000e+05  1.300000e+05   
4184  8400793  5.500000e+05  5.500000e+05   

                                               geometry  lake  
8     POLYGON ((109.74514 55.86611, 109.74541 55.865...     1  
9     POLYGON ((74.98387 46.83114, 74.98618 46.82939...     1  
10    POLYGON ((59.59419 46.31613, 59.59805 46.31584...     1  
82    POLYGON ((100.13228 75.18992, 100.14897 75.189...     1  
84    POLYGON ((89.38354 68.62044, 89.37909 68.61513...     1  
...                                                 ...   ...  
4176  POLYG

/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


           ID      area_cor      area_org  \
11    7000015  1.875152e+10  1.875152e+10   
12    7000016  6.716622e+10  6.716622e+10   
13    7000017  3.282665e+10  3.282665e+10   
14    7000018  2.954400e+10  2.954400e+10   
141   7000152  5.385340e+09  5.385340e+09   
...       ...           ...           ...   
4212  8416799  6.000000e+05  6.000000e+05   
4213  8417209  3.300000e+05  3.300000e+05   
4217  8418019  9.000000e+05  9.000000e+05   
4223  8420336  3.500000e+05  3.500000e+05   
4228  8423398  5.000000e+05  5.000000e+05   

                                               geometry  lake  
11    POLYGON ((13.33968 14.34845, 13.35371 14.34654...     1  
12    POLYGON ((31.91181 -2.72236, 31.91141 -2.72289...     1  
13    POLYGON ((29.18881 -3.34623, 29.19245 -3.34670...     1  
14    POLYGON ((34.58819 -13.87931, 34.58832 -13.880...     1  
141   POLYGON ((32.97069 23.41986, 32.97030 23.41953...     1  
...                                                 ...   ...  
4212  POLYG

...

           ID      area_cor      area_org  \
0     7000001  3.770019e+11  3.770019e+11   
7     7000010  1.744401e+10  1.744401e+10   
83    7000092  1.098170e+09  1.098170e+09   
85    7000094  1.063610e+09  1.063610e+09   
86    7000095  1.167600e+09  1.167600e+09   
...       ...           ...           ...   
4134  8361686  1.100000e+05  1.100000e+05   
4139  8363550  2.000000e+05  2.000000e+05   
4142  8365569  2.000000e+05  2.000000e+05   
4154  8369156  2.900000e+05  2.900000e+05   
4156  8371045  7.200000e+05  7.200000e+05   

                                               geometry  lake  
0     POLYGON ((49.96181 37.43847, 49.96457 37.44022...     1  
7     POLYGON ((30.88438 61.74966, 30.88663 61.74813...     1  
83    POLYGON ((28.57840 69.36283, 28.57953 69.36265...     1  
85    POLYGON ((33.33149 68.08159, 33.33645 68.07742...     1  
86    POLYGON ((34.12605 63.90383, 34.12912 63.90174...     1  
...                                                 ...   ...  
4134  POLYG

/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


           ID      area_cor      area_org  \
64    7000073  3.661100e+09  3.661100e+09   
65    7000074  1.066750e+09  1.066750e+09   
66    7000075  2.304630e+09  2.304630e+09   
67    7000076  2.349170e+09  2.349170e+09   
68    7000077  2.614960e+09  2.614960e+09   
...       ...           ...           ...   
4030  8110087  9.800000e+05  9.800000e+05   
4031  8111200  1.100000e+05  1.100000e+05   
4032  8112594  4.100000e+05  4.100000e+05   
4033  8112681  1.300000e+05  1.300000e+05   
4034  8113738  2.100000e+05  2.100000e+05   

                                               geometry  lake  
64    POLYGON ((-62.54764 7.80792, -62.54666 7.80761...     1  
65    POLYGON ((-54.95903 4.65625, -54.95884 4.66194...     1  
66    POLYGON ((-60.24708 -1.18875, -60.24583 -1.188...     1  
67    POLYGON ((-49.80958 -3.74958, -49.80895 -3.747...     1  
68    POLYGON ((-40.97361 -9.12584, -40.97445 -9.127...     1  
...                                                 ...   ...  
4030  POLYG

/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


           ID      area_cor      area_org  \
149   7000160  1.129030e+09  1.129030e+09   
163   7000175  8.026700e+09  8.026700e+09   
164   7000176  2.508030e+09  2.508030e+09   
165   7000177  3.539200e+09  3.539200e+09   
166   7000178  3.373980e+09  3.373980e+09   
...       ...           ...           ...   
4230  8425748  7.200000e+05  7.200000e+05   
4231  8426503  9.400000e+05  9.400000e+05   
4232  8426948  2.400000e+05  2.400000e+05   
4233  8426992  6.900000e+05  6.900000e+05   
4234  8427034  1.900000e+05  1.900000e+05   

                                               geometry  lake  
149   POLYGON ((98.56369 2.89268, 98.56569 2.89264, ...     1  
163   POLYGON ((137.16129 -27.81859, 137.16236 -27.8...     1  
164   POLYGON ((139.96762 -30.23810, 139.96666 -30.2...     1  
165   POLYGON ((137.21511 -30.17508, 137.21445 -30.1...     1  
166   POLYGON ((136.02525 -31.95851, 136.02559 -31.9...     1  
...                                                 ...   ...  
4230  POLYG